## Find Missing Bounding Box for UMedia Maps Metadata

If you have already got the metadata CSV file after running the `harvest.py` and found the **Bounding Box** field are empty for all records, you may need this script to help you find those missing Bounding Boxes. However, if you can find the Bounding Box values, then just igonore this script.

> Original created on Dec 1, 2020 by Gene Cheng

> Updated on May 1st, 2021 by Gene Cheng

> Updated on Nov 8th, 2021 by Gene Cheng

> Updated on Nov 22th, 2021 by Gene Cheng

### STEP 1. Go and Download the Lookup Sheet

In order to find the bounding box, we need to download a sheet which may includes both **Identifier** and **Coordinate** fields for the maps you harvested just now. Here's the [link](https://docs.google.com/spreadsheets/d/1A2MxmWxQ31_aDpxYQ5WSS9WcuoGdLGa9MOJWF-M7q6U/edit#gid=837746637) to a spreadsheet from Map Library and you can find there are more than one sheets.

Usually, I will copy some random `Identifiers` from the metadata CSV and use the **Find and Replace** tool from Google Sheet to search for these Identifiers among all exsiting sheets, and the one which contains these Identifiers is the sheet you need to download next. These spreadsheets are updated frequently, so you may need to download right before you execute this script.

After you download, please make sure to move it into the current directory and rename it to `bbox_lookup.csv` instead.


### STEP 2. Read the Lookup Sheet

Since we only need the **System Identifier** and the **Coordinates** fields from the lookup sheet, So we can first find the index of these two columns before reading the file by `index()`.

Then, read and extract these two fields only. Some values may have leading or trailing spaces, so that we need to use the `strip()` to remove them before store into a dictionary.

In [1]:
import csv
import re

In [61]:
bbox_lookup = {}
with open('bbox_lookup.csv') as fr:
    reader = csv.reader(fr)
    fields = next(reader)
    ID_index = fields.index('SYSTEM IDENTIFIER')
    coords_index = fields.index('Coordinates')
    for row in reader:
        ID = row[ID_index].strip()
        coords = row[coords_index].strip()
        bbox_lookup[ID] = coords

### STEP 3. Clean Data

This step might be the most important process through the whole script, since the coordinates values from the lookup sheet are 2 different formats of **Degree, Minute, Second** coordinates concatenated together which represent the same coordinates order by **West, East, North and South**. For example: `W0931944 W0931137 N0450304 N0445324; (W 93°19\'44"--W 93°11\'37"/N 45°03\'04"--N 44°53\'24")` 

So first of all, we will use the `clean_coords()` function to choose the first format`W0931944 W0931137 N0450304 N0445324` and separate them into `w,e,n,s` individually. 



In [48]:
# Select the first format of dms
# Separate into four coordiantes
def clean_data(val):
    coords = val.split(';')[0]
    coordlist = coords.split(' ')
    return coordlist


Then, we will continue using the `parse_dms()` function to parse each coordinate into **Direction, Degree, Minute and Second** separately. Take the coordinate `W0931944` as an example, the first letter represents the **Direction**, the next three letters represent the **3-digit Degree**, the following two letters represent the **2-digit Minute** and the last two letters stand for the **2-digit Second**


In [32]:
# Parse a DMS coordinate into different parts: direction, Degree, Minute, Second
def parse_dms(coord):
    dir = coord[0]
    deg = coord[1:4]
    min = coord[4:6]
    sec = coord[6:]
    return (dir, deg, min, sec)


Last, convert the coordinate from **Degree,Minute,Second** to **Decimal Degree** with 4 digits after decimal point.

In [35]:
# Convert a coordiante from DMS format to Decimal Degree
def dms2dd(parsed_dms):
    (direction, degree, minute, second) = parsed_dms
    dd = float(degree) + float(minute)/60 + float(second)/(60*60)
    if direction == 'S' or direction == 'W':
        dd *= -1
    return str(round(dd,4))
    

In [63]:
bbox_dd = {}
for ID,coords in bbox_lookup.items():  
    if ';' in coords:
        dds = []
        for coord in clean_data(coords):
            dd = dms2dd(parse_dms(coord))
            dds.append(dd)
        
        if len(dds) == 4:
            [w, e, n, s] = dds
            bbox = ','.join([w,s,e,n])
        else:
            bbox = ','.join(dds)
    else:
        bbox = ''
  
    
    bbox_dd[ID] = bbox
    

In [72]:
bbox_dd['UMN_ALMA:9973979307101701']

'-118.5,22.6667,-108.0,33.0'

### STEP 4. Find Missing BBOX for UMedia Maps

Before running the following cells, please **edit** the first cell of Step 4 to the change the filename of the metadata CSV file from reaccession.

If more than one csv files exist, change the filename and rerun the step 4&5 again.

In [82]:
# Hello, please edit here !!
filename = 'reports/dateAdded_202110.csv'

In [83]:
records = []
with open(filename) as fr:
    reader = csv.reader(fr)
    fields = next(reader)
    for row in reader:
        records.append(row)

In [84]:
for record in records:
    ID_index = fields.index('Identifier')
    bbox_index = fields.index('Bounding Box')
    ID = record[ID_index].strip()
    if ID in bbox_dd:
        record[bbox_index] = bbox_dd[ID]

### STEP 5. Write a new CSV

Write to a new CSV file with the same name to overwrite the previous one.

In [85]:
with open(filename, 'w') as fw:
    writer = csv.writer(fw)
    writer.writerow(fields)
    writer.writerows(records)